<a href="https://colab.research.google.com/github/Kornieks/A-B-test-/blob/main/Portfolio_project_A_B_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import statsmodels.api as sm

##Connecting to the dataset

In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """
WITH session_info AS (
SELECT
  s.date,
  s.ga_session_id,
  sp.country,
  sp.device,
  sp.continent,
  sp.channel,
  ab.test,
  ab.test_group
FROM `DA.ab_test` AS ab
JOIN `DA.session` AS s
ON ab.ga_session_id = s.ga_session_id
JOIN `DA.session_params` AS sp
ON ab.ga_session_id = sp.ga_session_id),


sessions_with_orders AS (
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT o.ga_session_id) AS order_cnt
FROM `DA.order` as o
JOIN session_info
ON o.ga_session_id = session_info.ga_session_id
GROUP BY
session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group),


events AS (
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name,
  COUNT (ep.ga_session_id) AS event_cnt
FROM `DA.event_params` AS ep
JOIN  session_info
ON ep.ga_session_id = session_info.ga_session_id
GROUP BY  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name),


sessions AS (
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT ga_session_id) as session_cnt
FROM session_info
GROUP BY  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group),


new_accounts AS (SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT acs.ga_session_id) as new_account_cnt
FROM `DA.account_session` AS acs
JOIN session_info
ON acs.ga_session_id = session_info.ga_session_id
GROUP BY session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group)






SELECT
  sessions_with_orders.date,
  sessions_with_orders.country,
  sessions_with_orders.device,
  sessions_with_orders.continent,
  sessions_with_orders.channel,
  sessions_with_orders.test,
  sessions_with_orders.test_group,
  'sessions_with_orders' AS event_name,
  sessions_with_orders.order_cnt AS value
FROM sessions_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  events.event_name as event_name,
  events.event_cnt AS value
FROM events
UNION ALL
SELECT
  sessions.date,
  sessions.country,
  sessions.device,
  sessions.continent,
  sessions.channel,
  sessions.test,
  sessions.test_group,
  'sessions' AS event_name,
  sessions.session_cnt as value
FROM sessions
UNION ALL
SELECT
  new_accounts.date,
  new_accounts.country,
  new_accounts.device,
  new_accounts.continent,
  new_accounts.channel,
  new_accounts.test,
  new_accounts.test_group,
  'new_accounts' AS event_name,
  new_accounts.new_account_cnt AS value
FROM new_accounts
"""

In [ ]:
query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()

In [ ]:
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-12-08,Palestine,desktop,Asia,Direct,4,2,new_accounts,1
1,2020-12-08,Palestine,desktop,Asia,Direct,3,2,new_accounts,1
2,2020-11-06,Puerto Rico,desktop,Americas,Social Search,2,2,new_accounts,1
3,2020-11-06,Puerto Rico,desktop,Americas,Social Search,1,1,new_accounts,1
4,2020-12-08,Croatia,desktop,Europe,Direct,4,2,new_accounts,1


In [ ]:
data = df.groupby(['test','test_group','event_name'])['value'].sum().reset_index()
data

,test,test_group,event_name,value
0,1,1,add_payment_info,1988
1,1,1,add_shipping_info,3034
2,1,1,add_to_cart,1395
3,1,1,begin_checkout,3784
4,1,1,click,368
...,...,...,...,...
147,4,2,user_engagement,273633
148,4,2,view_item,98928
149,4,2,view_item_list,6
150,4,2,view_promotion,51985


In [ ]:
# Creating list of metrics in the format (numerator_event, denominator_event)
metrics = [
    ('add_payment_info', 'sessions'),
    ('add_shipping_info', 'sessions'),
    ('begin_checkout', 'sessions'),
    ('new_accounts', 'sessions'),
    ('sessions_with_orders', 'sessions'),
    ('view_promotion', 'sessions'),
    ('select_promotion', 'sessions'),
    ('select_promotion', 'view_promotion')
    ]

# List to store the final result
result = []

# Loop over each uniue test
for test in data['test'].unique():

  # Filter data for the current test
  df_data_test = data[data['test'] == test]

  # Pivot data
  df_data_test_pivot = (df_data_test.pivot(index='test_group', columns='event_name', values='value').fillna(0))

  # Loop over each metric from our list
  for metric in metrics:
    numerator = metric[0]  # establishing numerator_event_name
    denominator = metric[1] # establishing denominator_event_name

    # Control group
    numerator_value_1 = df_data_test_pivot.loc[1, numerator] # getting the count of the numerator event (like add_payment_info) for the first group (control)
    denominator_value_1 = df_data_test_pivot.loc[1, denominator] # getting the count of the denominator event (like sessions) for the first group (control)
    conversion_rate_control = numerator_value_1 / denominator_value_1 # conversion rate

    # Test group
    numerator_value_2 = df_data_test_pivot.loc[2, numerator]
    denominator_value_2 = df_data_test_pivot.loc[2, denominator]
    conversion_rate_test = numerator_value_2 / denominator_value_2

    #  Relative difference between test and control in percentage
    metric_rate_difference = (conversion_rate_test - conversion_rate_control) * 100 / conversion_rate_control

     # Z-test for statistical significance of the conversion difference
        #  arguments: number of successes (numerator) and number of observations (denominator)
    z_stat, p_value = sm.stats.proportions_ztest([numerator_value_2, numerator_value_1], [denominator_value_2, denominator_value_1])


    # Storing results in a dictionary
    result.append({
        'test': test,
        'metric': f'{numerator} / {denominator}',
        'numerator_event_name': numerator,
        'denomintor_event_name': denominator,
        'numerator_test_value': numerator_value_2,
        'denominator_test_value': denominator_value_2,
        'conversion_rate_test': conversion_rate_test,
        'numerator_control_value': numerator_value_1,
        'denomintor_control_value': denominator_value_1,
        'conversion_rate_control': conversion_rate_control,
        'metric_rate_%_difference': metric_rate_difference,
        'z_stat': z_stat,
        'p_value': p_value,
        'significance': 'significant' if p_value < 0.05 else 'not significant'
        })

In [ ]:
# Creating dataframe from the dictionary
df_result = pd.DataFrame(result)


In [ ]:
# Saving resuts in text file
df_result.to_csv('results.csv', index=False)

###[Tableau Dashboard](https://public.tableau.com/app/profile/kseniia.kornienko/viz/ABtestanaysis/FinalABtestanalysis?publish=yes)

###[CSV file](https://drive.google.com/file/d/16bFaSBnkYwtIPwYiylYDqgyhJ8YGtVC8/view?usp=sharing)



##Conclusions:
The goal of the project was to evaluate the impact of changes using A/B testing.
 Overall, the dataset contains four A/B tests. Users were split into Control (Group 1) and Test (Group 2) with an approximately equal traffic distribution (50/50), ensuring a fair comparison.
The analysis focused on understanding whether the test variant improved user behavior.

For each test and metric:
- conversion rates were calculated separately for control and test groups;
- the relative percentage difference between test and control was computed to quantify uplift or decline;
- results were aggregated into a structured dataset for reporting and visualization.

To validate whether observed differences were due to chance, a two-proportion z-test was applied.
A metric was considered statistically significant if p-value < 0.05.

This ensured that decisions were based not only on uplift size but also on statistical reliability.

Results were visualized in Tableau, including:
- group distributions;
- event-level differences;
- conversion rate comparisons;
- statistical significance summary.

Filters (date, channel, device, geography) enabled deeper exploratory analysis and stakeholder-friendly insights.

 *However, it is important to remember that due to the multiple comparison problem, statistical significance may not reliable across segments.*

**The Overall A/B test results show**

**Test #1**

Statistically significant uplifts were observed in:
Add payment info rate, from 4.38% to 4.93% (+12%)
Add shipping info rate, from 6.69% to 7.13% (+6.5%)
Begin checkout rate, from 8.34% to 8.90% (+6.6%)

The test variant positively impacted key mid-funnel metrics, while overall purchase conversion remained statistically unchanged.

**Test #2**

Statistically significant decrease was observed in view promotion rate, from 63.92% to 63.05% (-1.3%)

**Test #3**

Statistically significant downward trends were observed in view promotion rate, from 58.77% to 58.15% (-1%) and begin checkout rate, from 13.61% to 13.15% (-3.3%)

**Test #4**

Statistically significant falls were observed in:
View promotion rate, from 50.13% to 49.44% (-1.3%)
Begin checkout rate, from 11.95% to 11.67% (-2.3%)
New accounts rate, from 8.55% to 8.26% (-3.3%)


##Additional Exploratory analysis

### Z-test analysis by device, channel, continent

In [ ]:
data = df.groupby(['test','test_group','event_name','device','channel','continent'])['value'].sum().reset_index()
data

,test,test_group,event_name,device,channel,continent,value
0,1,1,add_payment_info,desktop,Direct,Africa,7
1,1,1,add_payment_info,desktop,Direct,Americas,129
2,1,1,add_payment_info,desktop,Direct,Asia,41
3,1,1,add_payment_info,desktop,Direct,Europe,48
4,1,1,add_payment_info,desktop,Direct,Oceania,2
...,...,...,...,...,...,...,...
10259,4,2,view_search_results,tablet,Social Search,Asia,7
10260,4,2,view_search_results,tablet,Social Search,Europe,3
10261,4,2,view_search_results,tablet,Undefined,Americas,3
10262,4,2,view_search_results,tablet,Undefined,Asia,1


In [ ]:

metrics = [
    ('add_payment_info', 'sessions'),
    ('add_shipping_info', 'sessions'),
    ('begin_checkout', 'sessions'),
    ('new_accounts', 'sessions'),
    ('sessions_with_orders', 'sessions'),
    ('view_promotion', 'sessions'),
    ('select_promotion', 'sessions'),
    ('select_promotion', 'view_promotion')
    ]

# Creating unique combinations of parameters
df_test_params = data[['test', 'device', 'channel', 'continent']].drop_duplicates()
print(data.head())
print(df_test_params.head())

result_3 = []

# Loop over each unique row
for idx, row in df_test_params.iterrows():

  # Filter data for the current row
  df_data_test = data[(data['test'] == row['test'])
                      & (data['device'] == row['device'])
                      & (data['channel'] == row['channel'])
                      & (data['continent'] == row['continent'])]

  df_data_test_pivot = (df_data_test.pivot(index='test_group', columns='event_name', values='value').fillna(0))

  if df_data_test_pivot.shape[0] != 2: # should be two groups
    print(f'Less than two groups for {row=}')
    continue

  for metric in metrics:
    numerator = metric[0]
    denominator = metric[1]

    # Check to prevent the code from crushing if a certain event column is missing in the data
    if numerator not in df_data_test_pivot.columns:
      print(f"Could not find {numerator} column in data")
      continue
    if denominator not in df_data_test_pivot.columns:
      print(f"Could not find {denominator} column in data")
      continue

    numerator_value_1 = df_data_test_pivot.loc[1, numerator]
    denominator_value_1 = df_data_test_pivot.loc[1, denominator]
    conversion_rate_control = numerator_value_1 / denominator_value_1

    numerator_value_2 = df_data_test_pivot.loc[2, numerator]
    denominator_value_2 = df_data_test_pivot.loc[2, denominator]
    conversion_rate_test = numerator_value_2 / denominator_value_2

    # Check to prevent the code from crushing if the values is 0 and could not be divided
    if denominator_value_1 == 0 or denominator_value_2 == 0:
      print(f"Could not calculate conversion rate for {row=}")
      continue

    # Filter out small sample sizes
    if denominator_value_1 < 100 or denominator_value_2 < 100:
      print(f"Too little data")
      continue

    metric_rate_difference = (conversion_rate_test - conversion_rate_control) * 100 / conversion_rate_control

    z_stat, p_value = sm.stats.proportions_ztest([numerator_value_2, numerator_value_1], [denominator_value_2, denominator_value_1])


    result_3.append({
        'test': row['test'],
        'device': row['device'],
        'channel': row['channel'],
        'continent': row['continent'],
        'metric': f'{numerator} / {denominator}',
        'numerator_event_name': numerator,
        'denomintor_event_name': denominator,
        'numerator_test_value': numerator_value_2,
        'denominator_test_value': denominator_value_2,
        'conversion_rate_test': conversion_rate_test,
        'numerator_control_value': numerator_value_1,
        'denomintor_control_value': denominator_value_1,
        'conversion_rate_control': conversion_rate_control,
        'metric_rate_%_difference': metric_rate_difference,
        'z_stat': z_stat,
        'p_value': p_value,
        'significance': 'significant' if p_value < 0.05 else 'not significant'
        })

   test  test_group        event_name   device channel continent  value
0     1           1  add_payment_info  desktop  Direct    Africa      7
1     1           1  add_payment_info  desktop  Direct  Americas    129
2     1           1  add_payment_info  desktop  Direct      Asia     41
3     1           1  add_payment_info  desktop  Direct    Europe     48
4     1           1  add_payment_info  desktop  Direct   Oceania      2
   test   device channel continent
0     1  desktop  Direct    Africa
1     1  desktop  Direct  Americas
2     1  desktop  Direct      Asia
3     1  desktop  Direct    Europe
4     1  desktop  Direct   Oceania
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too li

/tmp/ipython-input-841631252.py:64: RuntimeWarning: divide by zero encountered in scalar divide
  metric_rate_difference = (conversion_rate_test - conversion_rate_control) * 100 / conversion_rate_control


Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Could not find select_promotion column in data
Could not find select_promotion column in data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Too little data
Could not find select_promotion column in data
Could not find select_promotion column in data
Too little data
Too little data
Too little data
Too little data
Too little d

/tmp/ipython-input-841631252.py:64: RuntimeWarning: divide by zero encountered in scalar divide
  metric_rate_difference = (conversion_rate_test - conversion_rate_control) * 100 / conversion_rate_control


In [ ]:
df_result_detailed = pd.DataFrame(result_3)


# Bonferroni correction to adjust for a multiple comparison problem

n_tests = len(df_result_detailed)
print(n_tests)

alpha_adj = 0.05 / n_tests

df_result_detailed['p_value_adj'] = (df_result_detailed['p_value'] * n_tests).clip(upper=1)

df_result_detailed['significance_adj'] = df_result_detailed['p_value_adj'].apply(
    lambda p: 'significant' if p < 0.05 else 'not significant'
)



1208


In [ ]:
df_result_detailed.to_csv('results_3.csv', index=False)

In [ ]:
df_result_detailed.significance.value_counts()

,count
significance,
not significant,953
significant,255


In [ ]:
df_result_detailed.significance_adj.value_counts()

,count
significance_adj,
not significant,1173
significant,35
